In [ ]:
# Path para diretório raiz do projeto
import sys

sys.path.insert(0, sys.path[0].removesuffix("/src/jupyter"))
# Libs do projeto
import src.utils as utils
import pandas as pd
import numpy as np

# All necessary plotly libraries
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
data_file_list = utils.get_data_files()

acoes_prob_cair = pd.DataFrame()
acoes_prob_subir = pd.DataFrame()
for data_file in data_file_list:
    data = pd.read_csv(data_file, sep=";")
    data.dropna(inplace=True)
    if data.shape[0] > 0:
        data["previsao"] = "ESTAVEL"
        data["previsao"] = np.where(
            (data["adj_close"] > data["ema_close_200p"]) & (data["rsi"] > 70.0),
            "CAIR",
            data["previsao"],
        )
        data["previsao"] = np.where(
            (data["adj_close"] < data["ema_close_200p"]) & (data["rsi"] < 30.0),
            "SUBIR",
            data["previsao"],
        )
        data.to_csv(data_file, sep=";", index=False)

        if data.tail(1).previsao.values[0] == "CAIR":
            acoes_prob_cair = pd.concat([acoes_prob_cair, data.tail(1)])
        elif data.tail(1).previsao.values[0] == "SUBIR":
            acoes_prob_subir = pd.concat([acoes_prob_subir, data.tail(1)])

In [ ]:
print(f"Ações Probabilidade Cair: ")
acoes_prob_cair

In [ ]:
print(f"Ações Probabilidade Subir: ")
acoes_prob_subir

In [ ]:
filter_date = '	2022-01-01'

In [ ]:
print('Graficos Ações Probabilidade Subir')
fig_list = []
for ticker in acoes_prob_subir.ticker.unique():
	ticker_type = data.tail().type.values[0]
	title = f'{ticker_type.upper()}: {ticker} - Probabilidade Subir'
	data = pd.read_csv(f"{utils.stocks_dir}/{ticker}.csv", sep=";")
	data = data[data['open_time'] >  filter_date]
	fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3], specs=[[{"secondary_y": True}], [{"secondary_y": True}]])
	
	# fig.add_trace(go.Candlestick(x=data['s_open_time'], open=data['open'], high=data['high'], low=data['low'], close=data['close'], name='Price'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['close'], name='Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['adj_close'], name='Adj Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_close_200p'], name='EMA Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_adj_close_200p'], name='EMA Adj Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)

	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['rsi'], name='rsi', line_color='purple'), row=2, col=1, secondary_y=False)
	fig.add_trace(go.Bar(x=data['s_open_time'], y=data['volume'], name='volume', marker_color='red'), row=2, col=1, secondary_y=True)
	
	fig.add_hline(y=30, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hline(y=70, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hrect(y0=30, y1=70, line_width=0, fillcolor="blue", opacity=0.1, row=2)
	
	fig.update_layout(title_text=title, height=800, )
	fig.update(layout_xaxis_rangeslider_visible=False)
	fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "sun"])])

	fig_list.append({'fig': fig, 'title': f'{title}' })
	

for fig in fig_list:
	print(fig['title'])
	fig['fig'].show()

In [ ]:
print('Graficos Ações Probabilidade Cair')
fig_list = []
for ticker in acoes_prob_cair.ticker.unique():
	ticker_type = data.tail().type.values[0]
	title = f'{ticker_type.upper()}: {ticker} - Probabilidade Cair'
	data = pd.read_csv(f"{utils.stocks_dir}/{ticker}.csv", sep=";")
	data = data[data['open_time'] > filter_date]
	fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3], specs=[[{"secondary_y": True}], [{"secondary_y": True}]])
	
	# fig.add_trace(go.Candlestick(x=data['s_open_time'], open=data['open'], high=data['high'], low=data['low'], close=data['close'], name='Price'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['close'], name='Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['adj_close'], name='Adj Close'), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_close_200p'], name='EMA Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)
	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['ema_adj_close_200p'], name='EMA Adj Close 200p', line_shape='spline', line_smoothing=1.3), row=1, col=1)

	fig.add_trace(go.Scatter(x=data['s_open_time'], y=data['rsi'], name='rsi', line_color='purple'), row=2, col=1, secondary_y=False)
	fig.add_trace(go.Bar(x=data['s_open_time'], y=data['volume'], name='volume', marker_color='red'), row=2, col=1, secondary_y=True)
	
	fig.add_hline(y=30, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hline(y=70, row=2, line_width=1, line_dash="dash", line_color='black')
	fig.add_hrect(y0=30, y1=70, line_width=0, fillcolor="blue", opacity=0.1, row=2)
	
	fig.update_layout(title_text=title, height=800, )
	fig.update(layout_xaxis_rangeslider_visible=False)
	fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "sun"])])

	fig_list.append({'fig': fig, 'title': f'{title}' })
	

for fig in fig_list:
	print(fig['title'])
	fig['fig'].show()